# Generate point-in-time best execution data using dedicated service - C#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `point-in-time` approach and designed to highlight individual trades within market activity over a look back period. 

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the interval approach, suitable to analyze market activity over a look back period. Please refer to <i>bestex-interval</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *point-in-time* results.

### Inputs/outputs
Best execution PIT sample requires instrument's identifier, **a list of trades** and time window as per inputs. Results are as follows:
* a snapshot of occured trades around the given trade based on the input time window
* plot the request result and export data

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***


# Run BestEx Point-in-time sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.33.*-pre*"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: CsvHelper"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Plot charts
using XPlot.Plotly;
using XPlot.Plotly.Interactive;
// Import/export csv files 
using CsvHelper;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data

#### 3.1 Input trades import
We assume our trades are in a *csv file* that we imported in the same folder as the current data sample.

We aim to to ensure that it was the best possible execution for each and every trade. To do so, we have to retrieve the *trades as a market snapshot* and mark out the *individual trades*.

The following code snippets import *individual trades* from the *csv file*:

In [ ]:
// Input trade to load from the csv file
public class InputTrade
{
    public DateTime Timestamp { get; set; }
    public double Price { get; set; }
    public long Size { get; set; }
    public string Flag { get; set; }
}

In [ ]:
// Load the trades from the csv input file
var records = new List<InputTrade>();
using (var reader = new StreamReader("input/input_trades.csv"))
using (var csv = new CsvReader(reader, CultureInfo.InvariantCulture))
{
    records = csv.GetRecords<InputTrade>().ToList();
}

// Sort the trades by time stamp
var inputTrades = records.OrderBy(t => t.Timestamp).ToArray();

// Display the loaded trades
display(inputTrades); 

### Step 3: Create and process request
To request *tick trades* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick trades request parameters

#### 3.1 Instrument selection

In [ ]:
// Set the instrument  identifier: a ticker and an exchange code
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Best execution interval selection

In [ ]:
// Set the time window to delimit the input trade in seconds
var window = TimeSpan.FromSeconds(5 * 60);

#### 3.3 Time period delimitation

The following code snippets delimits the best execution time interval around each input trade based on the previously chosen time window.

In [ ]:
// Get the first and the last dates from the input trades
var firstDate = inputTrades.First().Timestamp.Date;
var lastDate = inputTrades.Last().Timestamp.Date;

// Build the tick trades date interval (we are using Google date time format)
var dateIntervals = new DateInterval()
{ 
    StartDate = new Date { Year = firstDate.Year, Month = firstDate.Month, Day = firstDate.Day },
    EndDate = new Date { Year = lastDate.Year, Month = lastDate.Month, Day = lastDate.Day }
};

// Get the first and the last timestamps from the input trades
var firstTime = inputTrades.Min(t => t.Timestamp.TimeOfDay).Add(-window);
var lastTime = inputTrades.Max(t => t.Timestamp.TimeOfDay).Add(window);

// Build the tick trades time interval (we are using Google date time format)
var timeInterval = new TimeInterval() 
{ 
    StartTime = new TimeOfDay { Hours = firstTime.Hours, Minutes = firstTime.Minutes, Seconds = firstTime.Seconds },
    EndTime = new TimeOfDay { Hours = lastTime.Hours, Minutes = lastTime.Minutes, Seconds = lastTime.Seconds }
};

#### 3.4 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

In [ ]:
// Generate the tick trades request
var request = new TickTradesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick trades service
var service = new TickTradesService.TickTradesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve occured trades

In [ ]:
// Get the trades 
// store the list of trades
var trades = new List<Tuple<Timestamp, Trade>>();

// Call the service
var call = service.TickTrades(request, headers);

// Process the responses
await foreach (var current in call.ResponseStream.ReadAllAsync())
{
    if(current.Mapping != null)
    {
        // First response contains the mapping fields
        // Skip the mapping data
        continue;
    }

    // Get the time stamp for the current trade
    var ts = current.Data.TimeStamp;

    // Trade
    if (current.Data.Trade != null) 
    {      
        trades.Add(new Tuple<Timestamp, Trade>(ts, current.Data.Trade));
    }
}

In [ ]:
// Display the trades
//--> Note : the time stamp is displayed by default (using the unix format)
//--> We can use other displayers for different output formats 
//--> For more details about the time stamp: https://developers.google.com/protocol-buffers/docs/reference/google.protobuf#google.protobuf.Timestamp 
//display(trades);

#### 4.3 Plot individual trades within market snapshot
Display trade prices and highlight the individual trades over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.
Plot a simple line graph with the trade prices and mark out the individual trades as follows: 

In [ ]:
// Create the time series for the graph
var market = new Dictionary<DateTime, double>();
foreach (var trade in trades) 
{
    market[trade.Item1.ToDateTime()] = trade.Item2.Price;
}

var input = new Dictionary<DateTime, double>();
foreach (var inputTrade in inputTrades) 
{
    input[inputTrade.Timestamp] = inputTrade.Price;
}

In [ ]:
// Create the graph
var bestexPitList = new List<Scattergl>
{
    new Scattergl
    { name = "Market trades", x = market.Keys, y = market.Values},
    new Scattergl
    { name = "Input trades", x = input.Keys, y = input.Values, mode="markers"}
};
var chart = Chart.Plot(bestexPitList);
chart.WithTitle($"Market snapshot and input trades for {ticker}");
chart.WithXTitle($"Time");
chart.WithYTitle($"Price");
chart.Width = 1500;
chart.Height = 800;
display(chart);

#### 4.4 Export market snapshot file

In [ ]:
// Export market and input trades to a csv file
var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
using (var writer = new StreamWriter($"output/{ticker}_bestex_pit.csv"))
using (var csv = new CsvWriter(writer, CultureInfo.InvariantCulture))
{
    // Store the market and input data into the snapshot list
    var snapshot = new List<InputTrade>();

    // First loop all market trades
    foreach (var trade in trades)
    {
        var flag = string.Empty;
        var keep = false;
        var time = trade.Item1.ToDateTime();
        
        // Check if the current market trade belongs to the interval (window)
        // Then check if the trade is our input trade
        foreach (var inputTrade in inputTrades)
        {
            var start = inputTrade.Timestamp.Add(-window);
            var end = inputTrade.Timestamp.Add(window);
            
            if (time > start && time < end)
            {
                keep = true;
                
                // Check if the trade is our input trade
                if((time == inputTrade.Timestamp) && (trade.Item2.Price == inputTrade.Price) && (trade.Item2.Size == inputTrade.Size)) 
                {
                    flag = "INPUT";
                    goto store; 
                }  
            }   
        }
        
        store:
        if(keep)
        {
            snapshot.Add(new InputTrade(){Timestamp = time,Price = trade.Item2.Price, Size = trade.Item2.Size,Flag = flag });
        }
    }
    
    // Save csv file
    csv.WriteRecords(snapshot);
}